In [1]:
import numpy as np
import os
import time 
from math import sqrt 
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from numpy import pi
from Read_input import ReadInput
from img_to_etas import Etas
import pickle
start_time = time.time()

path = os.getcwd()

In [2]:
dt = .008
kappa = 0.3
L = 20

Nx = 128
Ny= 128
dx = 0.5
dy = 0.5

num_regions = 2

ntimesteps = 10000
nprint = 50
coefA = 1.0

In [3]:
class AC:
    def __init__(self, Nx, Ny, num_regions, dx, dy):
        self.Nx = Nx
        self.Ny = Ny
        self.NxNy = Nx*Ny
        self.nog = num_regions
        self.cx = Nx/2
        self.cy = Ny/2
        self.kxx = 2*pi*np.fft.fftfreq(Nx, d=dx)
        self.kyy = 2*pi*np.fft.fftfreq(Ny, d=dy)
        self.kx = np.append(self.kxx, 0)
        self.ky = np.append(self.kyy, 0)
        
    def k2_k4 (self):
        k2 = np.zeros((self.Nx, self.Ny))
        for i in range(self.Nx):
            for j in range(self.Ny):
                k2[i,j] = self.kx[i]*self.kx[i]+self.ky[j]*self.ky[j]
        k4=k2*k2
        return k4, k2
    def get_images(self):
        #Red all images form input folder 
        s = ReadInput()
        images = s.read()
        return images
def free_energy(etas, eta, igrain, Nx, Ny, ngrain):
    
    A = 1.0
    B= 1.0
    summ = np.zeros((Nx, Ny))
    for jgrain in range(ngrain):
        if (jgrain != igrain ):
            one = etas[:,:,jgrain]
            two = np.power(one, 2)
            three = np.power(one, 3)
            summ = np.add(summ, two)
    
    dfdeta = A*(2.0*B*np.multiply(eta, summ) + np.power(eta, 3) - eta)
    fre_eng = (-A/2)*eta**2+(B/4)*eta**4+eta**2*summ
    testfor = -A*eta+B*eta**3+2*eta*summ
    return dfdeta, fre_eng, testfor
    
def plot(Nx, Ny, eta3, step):
    eta_r = np.reshape(eta3, (Nx, Ny))
    plt.imsave('%d.png'%step, eta_r, cmap='gray')


In [4]:
model = AC(Nx, Ny, num_regions, dx, dy)
k4, k2 = model.k2_k4()
all_images = model.get_images()
os.chdir("..")

In [5]:
print(len(all_images))

1


In [6]:

counter=0 
   
f_val = []
checkk = []
full = np.zeros((1, 201, 128, 128, 2))

full[:,:,:,:,1]=1

ii =0

In [8]:
for img in all_images:
    print(ii)
    jj=0

    s = Etas(img, num_regions)
    etas = s.create_etas()

    counter +=1
    is_all_one = True
    glist = np.ones(num_regions)
    dt = 0.008
    feneg = []
    for step in tqdm (range(ntimesteps), desc = 'tqdm() Progress Bar'):
        for igrain in range(num_regions):
            if (glist[igrain] == 1):
                
                eta = etas [:,:, igrain]
                dfdeta, fre_eng, testfor = free_energy(etas, eta, igrain, Nx, Ny, num_regions)
                #print(dfdeta)
               # print(testfor)
                etak = np.fft.fft2(eta)
                dfdetak = np.fft.fft2(dfdeta)
                denom = 1 + dt * coefA*L*kappa * k2
                number = dt * L * dfdetak
                etak = (etak -number)/denom
                eta = np.fft.ifft2 (etak)
                eta = eta.real
                eta = np.where(eta>0.9999, 0.9999, eta)
                eta = np.where (eta<=0.00001, 0.00001, eta)
                etas[:,:,igrain] = eta[:,:]
                
                sss = 0
                checks = sum(eta)
                sss = sum(checks)
                sss = sum(checks)/(Nx*Ny)
                if (sss<0.001):
                    glist[igrain]=0
                    # print("this grain has been eliminated:%d \n" %igrain)
                    # etas[:,:,0]=0.0
                    # etas[:,:,1]=1.0
                    is_all_one = False
                    break
        if step%nprint == 0 or step == ntimesteps-1:
            # print("\n %d and %d and step %d"%(ii, jj, step))
            
            if is_all_one:
            
                full[ii, jj, :,:,:] = etas
                jj+=1
            
    ii+=1

0
Unique labels in label dataset are:  [0 1]


tqdm() Progress Bar: 100%|██████████████████████████████████████████████████████| 10000/10000 [00:48<00:00, 205.70it/s]


In [9]:
end_time = time.time()
etime=-(start_time-end_time)
print ("\n compute time is :%1d seconds"%etime) 
full = np.float32(full)


 compute time is :196 seconds


In [17]:
mat= full[0,1,:,:,1]

In [21]:
for i in range (shape[1]):
    mat = full[0, i , :,:,1]
    plt.imsave('%d.png'%i, mat, cmap='gray')
    